# ORGANISM: Escherichia_coli
# Datasource: https://downloads.thebiogrid.org/BioGRID/Release-Archive/BIOGRID-3.5.182/

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df_original = pd.read_csv("data/BIOGRID-ORGANISM-Escherichia_coli_K12_W3110-3.5.182.tab2.txt", sep='\t')

/home/ranap/anaconda3/envs/MLenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_original.head()

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,Pubmed ID,Organism Interactor A,Organism Interactor B,Throughput,Score,Modification,Phenotypes,Qualifications,Tags,Source Database
0,1108752,832467,12931104,17742,4260098,AT5G24020,Y75_p1148,MIND,minC,ACCUMULATION AND REPLICATION OF CHLOROPLASTS 1...,...,19457228,3702,316407,Low Throughput,-,-,-,-,-,BIOGRID
1,1108754,12931104,832467,4260098,17742,Y75_p1148,AT5G24020,minC,MIND,-,...,19457228,316407,3702,Low Throughput,-,-,-,-,-,BIOGRID
2,1108755,12931104,12931104,4260098,4260098,Y75_p1148,Y75_p1148,minC,minC,-,...,19457228,316407,316407,Low Throughput,-,-,-,-,-,BIOGRID
3,1108756,832467,12931104,17742,4260098,AT5G24020,Y75_p1148,MIND,minC,ACCUMULATION AND REPLICATION OF CHLOROPLASTS 1...,...,19457228,3702,316407,Low Throughput,-,-,-,-,-,BIOGRID
4,1112840,12933092,12933505,4262086,4262499,Y75_p2558,Y75_p3714,ffh,ftsY,-,...,20385832,316407,316407,Low Throughput,-,-,-,-,-,BIOGRID


In [4]:
df = df_original[["Official Symbol Interactor A", "Official Symbol Interactor B"]]

In [5]:
df.head()

,Official Symbol Interactor A,Official Symbol Interactor B
0,MIND,minC
1,minC,MIND
2,minC,minC
3,MIND,minC
4,ffh,ftsY


In [6]:
G = nx.from_pandas_edgelist(df, 'Official Symbol Interactor A', 'Official Symbol Interactor B', create_using=nx.DiGraph())

In [7]:
print(G.is_directed(), len(G.nodes), len(G.edges()))

True 4063 182911


In [8]:
# deleting self-loop edges
for n, nbrs in G.adj.items():
    if n in nbrs:
        G.remove_edge(n,n)

In [9]:
print(G.is_directed(), len(G.nodes), len(G.edges()))

True 4063 182910


### 3 nodes chain excluding common regulator 

In [10]:
def common_regulator(G, node1, node2):
    in_node1 = [a[0] for a in G.in_edges(node1)]
    in_node2 = [b[0] for b in G.in_edges(node2)]
    for i in in_node1:
        if i in in_node2:
            return True
        return False

In [11]:
common_regulator(G,'marA','putP')

False

In [12]:
node_set = set() # to save the node list
for node1 in G.nodes:
    for node2 in G.neighbors(node1):
        if (G.out_degree(node2)>1 or G.out_degree(node2)>1 or node2==node1):
            continue
        else:
            for node3 in G.neighbors(node2):
                if(node3 not in [node1, node2] and (node3 not in G.neighbors(node1)) and not(common_regulator(G, node1, node3))): 
                    print(node1, node2, node3)
                    node_set.add(node1); node_set.add(node2); node_set.add(node3);

minC rpoE lgt
ffh psd slt
ftsY rpoB alsB
ftsY rpoC alsB
yjiY yjgP yjgQ
yjiO rpoE lgt
yjiJ yjgP yjgQ
yjiJ rpoE lgt
yjhA yjgP yjgQ
yjhF rpoE lgt
ytfN yjgP yjgQ
ytfM rpoE lgt
ytfF yjgP yjgQ
yjeP yjgP yjgQ
yjeP rpoE lgt
yjeM yjgP yjgQ
yjeM rpoE lgt
frdC psd slt
sugE yjgP yjgQ
yjcR yjgP yjgQ
yjcR rpoE lgt
yjcE yjgP yjgQ
yjcD rpoE lgt
alr purA ytfN
alr malF ytfL
alr yjbB nrfA
alr rpoB alsB
alr uhpC uhpB
alr cysE mrr
alr pitB groS
alr hycC hycB
alr aroF ulaA
alr yfhG yfhB
alr sseB nrfC
alr yfcZ amiC
alr ubiG ggt
alr yeeI purC
alr yecE bcr
alr yniA gcvH
alr ydiU hypE
alr lsrC secY
alr ycdM rbsA
alr tauD ssuA
tatB psd slt
tatB yecT rplA
pldB rpoE lgt
xerC yjgP yjgQ
xerC rpoE lgt
hemY rpoE lgt
rffA psd slt
rffC psd slt
rffC rpoE lgt
rffD dapF rsgA
glmU yjgP yjgQ
glmU rpoE lgt
glmS rpoE lgt
glmS ytfL ppa
glmS yjbB nrfA
bglH yjgP yjgQ
yieG yjgP yjgQ
yieG rpoE lgt
yieG purA ytfN
yicO yjgP yjgQ
spoT yjgP yjgQ
spoT rpoE lgt
ttk psd slt
rfaQ psd slt
rfaB psd slt
rfaI yjgP yjgQ
rfaI psd slt
rfaC yjgP y

lpxB yjgP yjgQ
lpxB psd slt
lpxD yjgP yjgQ
yaeT rpoE lgt
yaeT rbn yiiM
ispU psd slt
ispU yeiI yoaC
ispU ymfB ytfE
panC yjgP yjgQ
panC bglA yggV
murF yjgP yjgQ
murF rpoE lgt
folA yjgP yjgQ
fecB rpoE lgt
fecD rpoE lgt
fecD yjhU yjhH
yjbF rpoE lgt
glpF malF ytfL
fadB yjgP yjgQ
tatC yjgP yjgQ
tatA rpoE lgt
trxA yjgP yjgQ
trxA ytfL ppa
gidB yjgP yjgQ
gidB yjhU yjhH
mdtL yjgP yjgQ
yidK yjgP yjgQ
coaD rpoE lgt
yhjY rpoE lgt
bcsB yjgP yjgQ
yhiI rpoE lgt
ugpC yjgP yjgQ
frlB rpoE lgt
gspI rpoE lgt
gspE yjgP yjgQ
secY rpoE lgt
secY rpoB alsB
secY rpoC alsB
secY yjbB nrfA
acrF yhiO mdtF
rpmA yjgP yjgQ
rpmA rpoE lgt
ygiH rpoB alsB
ygiH rpoC alsB
bacA yjgP yjgQ
bacA psd slt
bacA yrbG yrbK
zapA psd slt
amiC metE dsbA
amiC purA ytfN
amiC rpoB alsB
amiC rpoC alsB
eno cpdB deoB
yfiH purA ytfN
yfiH adiA ytfL
yfiH glnG glnL
yfiH metE dsbA
yfiH cysE mrr
yfiH yraR yhbT
yfiH yqiC ytfN
yfiH qseC zntA
yfiH galP atpH
yfiH gudX exuT
yfiH ygbI glpD
yfiH gshA mioC
yfiH yfiQ pheA
yfiH evgS clpB
yfiH yfbM garK
yfiH 

ygcJ pcm tatC
ygcJ ygbI glpD
ygcJ ygaH emrA
ygcJ ygaM emrB
ygcJ yfjF ygjD
ygcJ yfhG yfhB
ygcJ yfeX mreC
ygcJ evgS clpB
ygcJ yeiL def
ygcJ gatZ mgtA
ygcJ ydiU hypE
ygcJ lsrC secY
ygcJ rhtA yecS
fliA yedV kduI
bolA purA ytfN
bolA yjdI yjdJ
bolA rpoC alsB
bolA ilvC purD
bolA yqjE yqjK
bolA uxaC yibK
bolA ygjR exuT
bolA yghG pppA
bolA gudX exuT
bolA pcm tatC
bolA yeiL def
bolA mlrA mepA
bolA yedV kduI
bolA flhB fliP
bolA yecT rplA
bolA chbC fliY
bolA lsrC secY
bolA ycaC oppA
bolA artP ybjP
bolA ybiC purM
bolA tesB ftsK
bolA ampE yeeJ
ppiC glnG glnL
ppiC yigA yjiA
ppiC uhpC uhpB
ppiC yicC purH
ppiC aldB acs
ppiC yraR yhbT
ppiC yqeB dacB
ppiC cysD amiC
ppiC yeiL def
ppiC ynfB rnhA
ppiC purK purF
sgbH garK rbbA
sgbH yqjC gor
sgbH uxaC yibK
sgbH qseC zntA
sgbH galP atpH
sgbH ygbI glpD
sgbH gatB yehU
sgbH yedI yjcE
sgbH ybjE fadD
livG phnJ phnH
livG rbsD ulaA
livG glgX yiaD
livG elbB purD
livG galP atpH
livG ygbI glpD
livG yfjF ygjD
livG hyfD mgtA
livG yfeX mreC
livG yfcA atpF
livG menD fucU
li

rbsA frvX yjeF
rbsA ygiW ppa
ydjJ phnJ phnH
ydjJ rpoB alsB
ydjJ eptB uhpA
ydjJ hdeB hdeA
ydjJ prkB recF
ydjJ cysD amiC
ydjJ yfjF ygjD
ydjJ ppx dut
ydjJ hyfD mgtA
ydjJ eutB eutE
ydjJ cysW yicO
ydjJ menD fucU
ydjJ yohF treF
ydjJ mlrA mepA
ydjJ gatD mgtA
ydjJ yeeE ygdL
ydjJ yedI yjcE
ydjJ yedF aceB
ydjJ tyrP ftsH
ydjJ osmE yhiD
ydjJ chbC fliY
ydjJ ydeI yghA
ydjJ cysZ degQ
gdhA fklB ytfN
gdhA rpoC alsB
gdhA rpoB alsB
gdhA yieF phnD
gdhA yhjC yhjE
gdhA yhiO mdtF
gdhA glgX yiaD
gdhA yhfA purD
gdhA prkB recF
gdhA yggL phnD
gdhA guaD hsdM
gdhA yqeB dacB
gdhA fucU yhbC
gdhA pcm tatC
gdhA ygbI glpD
gdhA srlB yjjU
gdhA yfjF ygjD
gdhA evgS clpB
gdhA hisJ trxA
gdhA lysP cadC
gdhA gatZ mgtA
gdhA yedF aceB
gdhA yecT rplA
gdhA chbC fliY
gdhA yniA gcvH
gdhA ydiK uvrD
gdhA ydhS ysgA
gdhA ydhI ydhJ
gdhA ydeH ydeI
gdhA lsrC secY
gdhA narI sppA
gdhA ybjX yedX
gdhA modB narL
gdhA purK purF
gdhA tesB ftsK
pspA uhpC uhpB
pspA glgX yiaD
pspA yfjF ygjD
pspA yfbM garK
pspA yecE bcr
pspA ynfB rnhA
pspA ydcI ydeI


ybcN yhdU yhfZ
ybcN yphF ytfB
ybcL purA ytfN
ybcL malE lamB
ppiB aspA efp
ppiB qor msrA
allC pyrB iadA
ybbP galP atpH
ybbN pyrB iadA
ybbK eptB uhpA
ybbK hyfD mgtA
ybaS hdeA gadA
copA malK ytfM
copA rpoE lgt
copA nagD kch
copA yjbB nrfA
ybaK yfiQ pheA
ushA gshA mioC
fsr proP frdA
recR ptsN prfC
ybaB ytfL ppa
dnaX glnG glnL
ybaX yraR yhbT
lon hisJ trxA
lon alsB yjjN
yajL yjdJ cysQ
xseB cpdB deoB
ribD ilvC purD
ribD ybiC purM
brnQ psd slt
phoR oxyR yjiZ
phoR cysD amiC
sbcD ymfB ytfE
mak uxaC yibK
yaiE pyrB iadA
tauA cysD amiC
tauA tauD ssuA
frmR frlR ugpB
betB yibF fpr
yagS yphF ytfB
intF ytfL ppa
ykfI yphF ytfB
dinB ytfL ppa
dinB yecE bcr
dinB rhtA yecS
dinB exuT xylE
yafV acs msrA
dnaQ adiA ytfL
rnhA yphF ytfB
metI speA ggt
metQ cpdB deoB
yaeQ yeiI yoaC
pyrH sbp deoA
map pyrB iadA
yaeI dusB rfe
yadH pyrB iadA
hpt nirB metH
aroP tnaA rbsB
ilvI nirB metH
tbpA purA ytfN
tbpA tnaA rbsB
djlA artP ybjP
pdxA moeB ispE
carB pyrB iadA
carA pyrB iadA
nhaA ytfL ppa
nhaA rbsD ulaA
dnaJ cpdB deoB
cs

In [13]:
# H = G.subgraph(list(node_set))
# pos = nx.circular_layout(H)
# nx.draw(H, pos, cmap = plt.get_cmap('jet'), node_size = 2000)
# nx.draw_networkx_labels(H, pos)
# plt.show()

### 4 nodes chain

In [15]:
# pattern 2 search
for node1 in G.nodes:
    for node2 in G.neighbors(node1):
        if (G.out_degree(node2)>1 or G.out_degree(node2)>1 or node2==node1):
            continue
        else:
            for node3 in G.neighbors(node2):
                if (G.out_degree(node3)>1 or G.out_degree(node3)>1 or node3 in [node1, node2]):
                    continue
                else:
                    for node4 in G.neighbors(node3):
                        if(node4 not in [node1, node2, node3] and (node4 not in G.neighbors(node1)) and not(common_regulator(G, node1, node4))):
                            print(node1, node2, node3, node4)

ftsY rpoB alsB yjjN
ftsY rpoC alsB yjjN
alr malF ytfL ppa
alr rpoB alsB yjjN
dppA rpoB alsB yjjN
dppA rpoC alsB yjjN
dppA pyrB iadA yjiM
agaS hdeB hdeA gadA
agaS ydeH ydeI yghA
exbB pyrB iadA yjiM
exbB yjdI yjdJ cysQ
exbB gudX exuT xylE
exbB shiA yjbB nrfA
exbB ydeH ydeI yghA
glpQ rpoB alsB yjjN
glpQ rpoC alsB yjjN
rfbC pyrB iadA yjiM
dacD rpoB alsB yjjN
dacD rpoC alsB yjjN
yeeO malF ytfL ppa
ddpC aldB acs msrA
oppA rpoC alsB yjjN
oppA rpoB alsB yjjN
oppA yedF aceB aceA
oppA yedO phnL fecB
ompA yedO phnL fecB
cydC rpoB alsB yjjN
cydC rpoC alsB yjjN
mrdA rpoB alsB yjjN
mrdA rpoC alsB yjjN
rof malF ytfL ppa
rof rpoC alsB yjjN
mraY rpoB alsB yjjN
mraY rpoC alsB yjjN
mreC rpoB alsB yjjN
mreC rpoC alsB yjjN
yicJ rpoB alsB yjjN
yicJ rpoC alsB yjjN
mdtB rpoB alsB yjjN
mdtB rpoC alsB yjjN
pgsA rpoB alsB yjjN
pgsA rpoC alsB yjjN
tap rpoB alsB yjjN
tap rpoC alsB yjjN
folD pyrB iadA yjiM
ampH rpoB alsB yjjN
ampH rpoC alsB yjjN
glpF malF ytfL ppa
secY rpoB alsB yjjN
secY rpoC alsB yjjN
ygiH rpoB a

### 5 nodes chain

In [16]:
# pattern 2 search
for node1 in G.nodes:
    for node2 in G.neighbors(node1):
        if (G.out_degree(node2)>1 or G.out_degree(node2)>1 or node2==node1):
            continue
        else:
            for node3 in G.neighbors(node2):
                if (G.out_degree(node3)>1 or G.out_degree(node3)>1 or node3 in [node1, node2]):
                    continue
                else:
                    for node4 in G.neighbors(node3):
                        if (G.out_degree(node4)>1 or G.out_degree(node4)>1 or node4 in [node1, node2, node3]):
                            continue
                        else:
                            for node5 in G.neighbors(node4):
                                if(node4 not in [node1, node2, node3, node4] and (node5 not in G.neighbors(node1)) and not(common_regulator(G, node1, node5))):
                                    print(node1, node2, node3, node4, node5)                            

Comment: No chain of length 5 found